# Data Preprocessing 

In [366]:
# nltk.download('wordnet')

In [1]:
import pandas as pd
import numpy as np

import re
import string
import nltk
import pickle
import sklearn
import scipy
from wordcloud import WordCloud
# Display the word cloud
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.probability import FreqDist
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

In [2]:
constraint_train=pd.read_csv("data/constraint_train.csv")
constraint_test=pd.read_csv("data/constraint_test.csv")
constraint_val=pd.read_csv("data/constraint_val.csv")

claim_fake1=pd.read_csv("data/ClaimFake-1.csv")
claim_fake2=pd.read_csv("data/ClaimFake-2.csv")

claim_real1=pd.read_csv("data/ClaimReal-1.csv")
claim_real2=pd.read_csv("data/ClaimReal-2.csv")
claim_real3=pd.read_csv("data/ClaimReal-3.csv")
claim_real4=pd.read_csv("data/ClaimReal-4.csv")

news_real1=pd.read_csv("data/NewsReal-1.csv")
news_real2=pd.read_csv("data/NewsReal-2.csv")
news_real3=pd.read_csv("data/NewsReal-3.csv")
news_real4=pd.read_csv("data/NewsReal-4.csv")

news_fake1=pd.read_csv("data/Newsfake-1.csv")
news_fake2=pd.read_csv("data/Newsfake-2.csv")
news_fake3=pd.read_csv("data/Newsfake-3.csv")
news_fake4=pd.read_csv("data/Newsfake-4.csv")

In [3]:
constraint_train

(6420, 3)

In [15]:
news_fake3.shape

(56, 15)

In [6]:
constraint_val.shape

(2140, 3)

## Data description

In [369]:
df_train.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [370]:
df_test.head()

,id,tweet,label
0,1,Our daily update is published. States reported...,real
1,2,Alfalfa is the only cure for COVID-19.,fake
2,3,President Trump Asked What He Would Do If He W...,fake
3,4,States reported 630 deaths. We are still seein...,real
4,5,This is the sixth time a global health emergen...,real


In [371]:
df_val.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [372]:
claim_fake1.head()

,Unnamed: 0,fact_check_url,news_url,title
0,100000,medicalnewstoday.com,https://www.medicalnewstoday.com/articles/coro...,"""Spraying chlorine or alcohol on the skin kill..."
1,100001,medicalnewstoday.com,https://www.medicalnewstoday.com/articles/coro...,"""Only older adults and young people are at risk"""
2,100002,medicalnewstoday.com,https://www.medicalnewstoday.com/articles/coro...,"""Children cannot get COVID-19"""
3,100003,medicalnewstoday.com,https://www.medicalnewstoday.com/articles/coro...,"""COVID-19 is just like the flu"""
4,100004,medicalnewstoday.com,https://www.medicalnewstoday.com/articles/coro...,"""Everyone with COVID-19 dies"""


In [373]:
claim_real1.head()

,Unnamed: 0,fact_check_url,news_url,title
0,100000,who.int,https://www.who.int/news-room/q-a-detail/q-a-o...,"""How large does a meeting or event need to be ..."
1,100001,who.int,https://www.who.int/news-room/q-a-detail/q-a-o...,"""Does WHO recommend that all international mas..."
2,100002,who.int,https://www.who.int/news-room/q-a-detail/q-a-o...,"""What factors should organizers and health aut..."
3,100003,who.int,https://www.who.int/news-room/q-a-detail/q-a-o...,"""What if my organization does not have the exp..."
4,100004,who.int,https://www.who.int/news-room/q-a-detail/q-a-o...,"""If we go ahead with an international mass gat..."


In [374]:
news_fake1.head()

,Unnamed: 0,type,fact_check_url,archive,news_url,news_url2,news_url3,news_url4,news_url5,title,newstitle,content,abstract,publish_date,meta_keywords
0,0,post,https://factcheck.afp.com/false-advice-refusin...,https://perma.cc/J4N6-39D5,https://www.facebook.com/photo.php?fbid=551960...,NaN,NaN,NaN,NaN,Facebook posts shared in at least three countr...,NaN,NaN,NaN,NaN,NaN
1,1,article,https://www.politifact.com/factchecks/2020/apr...,NaN,http://legis.wisconsin.gov/assembly/republican...,NaN,NaN,NaN,NaN,Wisconsin is Òclearly seeing a decline in COVI...,"""Wisconsin Legislature Takes Gov. Evers to Court""",speaker robin vos r rochester and senate major...,NaN,NaN,""""""
2,2,post,https://factcheck.afp.com/posts-claim-children...,https://perma.cc/V4HX-M2XJ,https://www.facebook.com/iAmJessenia/photos/a....,NaN,NaN,NaN,NaN,Facebook posts claim a child who is infected w...,NaN,NaN,NaN,NaN,NaN
3,3,post,https://checkyourfact.com/2020/04/20/fact-chec...,NaN,https://www.facebook.com/kokernagnews/photos/a...,NaN,NaN,NaN,NaN,IndiaÕs Ministry of Home Affairs banning citiz...,NaN,NaN,NaN,NaN,NaN
4,5,post,https://checkyourfact.com/2020/04/20/fact-chec...,NaN,https://www.facebook.com/photo.php?fbid=102193...,NaN,NaN,NaN,NaN,"42 Democratic senators, plus two Independents,...",NaN,NaN,NaN,NaN,NaN


## Integrate all constrait datasets

In [375]:
constraint_df = pd.concat([constraint_train, constraint_test, constraint_val],axis=0, ignore_index=True)

if "id" in constraint_df.columns:
    constraint_df = constraint_df.drop(columns=["id"])

In [376]:
constraint_df['label'] = constraint_df['label'].map({'real': 0, 'fake': 1})
constraint_df.rename(columns={'tweet': 'content'},inplace=True)

In [377]:
constraint_df.head()

,content,label
0,The CDC currently reports 99031 deaths. In gen...,0
1,States reported 1121 deaths a small rise from ...,0
2,Politically Correct Woman (Almost) Uses Pandem...,1
3,#IndiaFightsCorona: We have 1524 #COVID testin...,0
4,Populous states can generate large case counts...,0


In [378]:
constraint_df.shape

(10700, 2)

## integrate all claim datasets

In [379]:
claim_fake = pd.concat([claim_fake1, claim_fake2],axis=0, ignore_index=True)

In [380]:
claim_real=pd.concat([claim_real1,claim_real2,claim_real3,claim_real4],axis=0, ignore_index=True)

In [381]:
claim_fake["label"] = 1
claim_real["label"] = 0

In [382]:
claim_df = pd.concat([claim_fake, claim_real],axis=0, ignore_index=True)

In [383]:
claim_df.rename(columns={'title': 'content'},inplace=True)

In [384]:
claim_df.drop('Unnamed: 0', axis=1, inplace=True)

In [385]:
claim_df.head()

,fact_check_url,news_url,content,label
0,medicalnewstoday.com,https://www.medicalnewstoday.com/articles/coro...,"""Spraying chlorine or alcohol on the skin kill...",1
1,medicalnewstoday.com,https://www.medicalnewstoday.com/articles/coro...,"""Only older adults and young people are at risk""",1
2,medicalnewstoday.com,https://www.medicalnewstoday.com/articles/coro...,"""Children cannot get COVID-19""",1
3,medicalnewstoday.com,https://www.medicalnewstoday.com/articles/coro...,"""COVID-19 is just like the flu""",1
4,medicalnewstoday.com,https://www.medicalnewstoday.com/articles/coro...,"""Everyone with COVID-19 dies""",1


In [386]:
claim_df.shape

(518, 4)

## integrate all news datasets

In [387]:
news_real= pd.concat([news_real1, news_real2, news_real3,news_real4],axis=0, ignore_index=True)

In [388]:
news_fake= pd.concat([news_fake1,news_fake2,news_fake3,news_fake4],axis=0, ignore_index=True)

In [389]:
news_real["label"] = 0
news_fake["label"] = 1

In [390]:
news_fake = news_fake.drop(['archive', 'news_url2', 'news_url3', 'news_url4', 'news_url5'], axis=1)

In [391]:
news_df = pd.concat([news_real, news_fake],axis=0, ignore_index=True)

In [392]:
print(news_df.isnull().sum())

Unnamed: 0           0
type                43
fact_check_url       0
news_url            43
title                0
newstitle          482
content           1017
abstract          2341
publish_date      4276
meta_keywords      482
label                0
dtype: int64


In [393]:
news_df["content"] = news_df["title"].fillna('') + ' ' + news_df["newstitle"].fillna('') + ' ' + news_df["content"].fillna('') + ' ' + news_df["abstract"].fillna('')

In [394]:
news_df = news_df.drop(['Unnamed: 0',"type","title","newstitle","abstract","publish_date","meta_keywords"], axis = 1)

In [395]:
news_df.head()

,fact_check_url,news_url,content,label
0,healthline.com,https://www.healthline.com/health-news/coronav...,"Coronavirus Outbreak: Live Updates ""Coronaviru...",0
1,healthline.com,https://www.healthline.com/health-news/screen-...,"Screen Time Doesn’t Hurt Kids’ Social Skills, ...",0
2,healthline.com,https://www.healthline.com/health-news/heres-e...,Here’s Exactly Where We Are with Vaccines and ...,0
3,healthline.com,https://www.healthline.com/health-news/heres-w...,Here’s What Happens to the Body After Contract...,0
4,healthline.com,https://www.healthline.com/health-news/why-you...,Why You Should Be Skeptical of Early Results o...,0


In [396]:
news_df.shape

(5457, 4)

## Integrate news_df and claim_df datasets

In [397]:
two_df= pd.concat([claim_df,news_df],axis=0, ignore_index=True)

In [398]:
two_df = two_df[["content", "label"]]

In [399]:
two_df.head()

,content,label
0,"""Spraying chlorine or alcohol on the skin kill...",1
1,"""Only older adults and young people are at risk""",1
2,"""Children cannot get COVID-19""",1
3,"""COVID-19 is just like the flu""",1
4,"""Everyone with COVID-19 dies""",1


In [400]:
two_df.shape

(5975, 2)

## Integrate two_df and constraint_df dataset

In [401]:
df= pd.concat([two_df,constraint_df],axis=0, ignore_index=True)

In [402]:
df.head()

,content,label
0,"""Spraying chlorine or alcohol on the skin kill...",1
1,"""Only older adults and young people are at risk""",1
2,"""Children cannot get COVID-19""",1
3,"""COVID-19 is just like the flu""",1
4,"""Everyone with COVID-19 dies""",1


In [403]:
df.shape

(16675, 2)

# Basic Text Pre-Processing

Text preprocessing steps include a few essential tasks to further clean the available text data. It includes tasks like:-

1. Stop-Word Removal : In English words like a, an, the, as, in, on, etc. are considered as stop-words so according to our requirements we can remove them to reduce vocabulary size as these words don't have some specific meaning

2. Lower Casing : Convert all words into the lower case because the upper or lower case may not make a difference for the problem. And we are reducing vocabulary size by doing so.

3. Stemming : Stemming refers to the process of removing suffixes and reducing a word to some base form such that all different variants of that word can be represented by the same form (e.g., “walk” and “walking” are both reduced to “walk”).

4. Tokenization : NLP software typically analyzes text by breaking it up into words (tokens) and sentences.

Pre-processing of the text is not the main objective of this notebook that's why I am just covering a few basic steps in a brief

### snowballSteammer VS Porter Stemmer

In [404]:
import re
import string
import tldextract
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs, HTML tags, square brackets, backslashes
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    
    # Remove punctuations, numbers, and newlines
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', '', text)
    
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
   # Perform stemming using Snowball Stemmer
    stemmer = SnowballStemmer('english')
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    # return ' '.join(stemmed_tokens)
     # Initialize lemmatizer
    # lemmatizer = WordNetLemmatizer()
    
    # Perform lemmatization
    # lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(stemmed_tokens)

In [405]:
df['content']= df['content'].apply(preprocess_text)

In [406]:
df.head()

,content,label
0,spray chlorin alcohol skin kill virus bodi,1
1,older adult young peopl risk,1
2,children get covid,1
3,covid like flu,1
4,everyon covid die,1


## split a manual testing dataset

In [407]:
df_manual_testing=df.tail(10)
for i in range(16674,16654,-1):
    df.drop([i],axis=0,inplace=True)

In [408]:
df.shape

(16655, 2)

In [409]:
df_manual_testing.to_csv("data/df_manual_testing.csv")
df.to_csv("data/df.csv")